In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
#Make sure you update it before starting this notebook
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git


import sklearn
import sklearn.feature_extraction.text

#import scipy.cluster.hierarchy
import gensim#For topic modeling
import requests #For downloading our datasets
import numpy as np #for arrays
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import matplotlib.cm #Still for graphics

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook.
#Also you can ignore the warning, it
%matplotlib inline

In [15]:
df = pandas.read_csv('t\df_80.csv', index_col=0)

df70 = pandas.read_csv('t\df_70.csv', index_col=0)
df80 = pandas.read_csv('t\df_80.csv', index_col=0)
df90 = pandas.read_csv('t\df_90.csv', index_col=0)
df00 = pandas.read_csv('t\df_00.csv', index_col=0)
df10 = pandas.read_csv('t\df_10.csv', index_col=0)

dd = pandas.concat([df70, df80, df90, df00, df10])

In [3]:
#Apply our functions
df['tokenized_text'] = df['AB'].apply(lambda x: lucem_illud_2020.word_tokenize(x))
df['normalized_tokens'] = df['tokenized_text'].apply(lambda x: lucem_illud_2020.normalizeTokens(x))

In [4]:
ngTFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.5, 
                                                                 max_features=3000, min_df=3, 
                                                                 stop_words='english', 
                                                                 norm='l2')
groupsTFVects = ngTFVectorizer.fit_transform(df['AB'])

In [5]:
def dropMissing(wordLst, vocab):
    return [w for w in wordLst if w in vocab]

df['reduced_tokens'] = df['normalized_tokens'].apply(lambda x: dropMissing(x, ngTFVectorizer.vocabulary_.keys()))

In [6]:
dictionary = gensim.corpora.Dictionary(df['reduced_tokens'])

In [7]:
## create a list of tuples containing each token and its count 
## use the first half and save the remainder for testing 
corpus = [dictionary.doc2bow(text) for text in df['reduced_tokens']]

In [8]:
# serialize the corpus as a file and load it 
gensim.corpora.MmCorpus.serialize('abstract.mm', corpus)
abmm = gensim.corpora.MmCorpus('abstract.mm')

In [9]:
# correctly formatted corpus that can be used for topic modeling and induction 
senlda = gensim.models.ldamodel.LdaModel(corpus=abmm, id2word=dictionary, num_topics=20, alpha='auto', eta='auto')

In [10]:
ldaDF = pandas.DataFrame({
        'titles' : df['TI'],
        'topics' : [senlda[dictionary.doc2bow(l)] for l in df['reduced_tokens']]
    })

In [11]:
topicsProbDict = {i : [0] * len(ldaDF) for i in range(senlda.num_topics)}

for index, topicTuples in enumerate(ldaDF['topics']):
    for topicNum, prob in topicTuples:
        topicsProbDict[topicNum][index] = prob

for topicNum in range(senlda.num_topics):
    ldaDF['topic_{}'.format(topicNum)] = topicsProbDict[topicNum]

In [12]:
K = senlda.num_topics  # N documents, K topics
topic_labels = ['Topic #{}'.format(k) for k in range(K)]

topicsDict = {}
for topicNum in range(senlda.num_topics):
    topicWords = [w for w, p in senlda.show_topic(topicNum)]
    topicsDict['Topic_{}'.format(topicNum)] = topicWords

wordRanksDF = pandas.DataFrame(topicsDict)
wordRanksDF

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Topic_5,Topic_6,Topic_7,Topic_8,Topic_9,Topic_10,Topic_11,Topic_12,Topic_13,Topic_14,Topic_15,Topic_16,Topic_17,Topic_18,Topic_19
0,determinant,family,health,determinant,male,determinant,determinant,health,child,family,factor,health,aggression,child,behavior,determinant,determinant,age,health,behavior
1,health,factor,care,model,female,study,factor,determinant,group,child,mother,study,age,health,determinant,child,health,study,population,health
2,family,breast,determinant,health,determinant,factor,support,patient,factor,self,age,determinant,support,determinant,health,study,support,determinant,study,patient
3,high,health,group,patient,health,high,patient,high,determinant,birth,woman,factor,determinant,age,age,use,study,group,factor,age
4,analysis,feeding,family,female,study,age,effect,care,care,alcohol,study,patient,child,mortality,study,mother,contraceptive,health,change,determinant
5,activity,study,coverage,factor,effect,risk,use,medical,study,high,child,behavior,level,factor,level,variable,theory,woman,economic,result
6,study,child,study,result,experience,effect,health,activity,family,subject,determinant,model,dental,high,fertility,health,behavior,child,age,study
7,child,effect,high,child,woman,disease,variable,group,test,study,intake,status,difference,experience,model,factor,family,level,determinant,individual
8,model,age,class,support,behavior,increase,relationship,study,variable,factor,smoking,class,study,analysis,effect,woman,model,abortion,child,physical
9,important,sample,specific,subject,change,group,study,level,result,analysis,year,variable,factor,result,factor,paper,patient,population,effect,model
